In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://d33wubrfki0l68.cloudfront.net/4abebe27614d96661267aa3d020abab97adf4a5e/4e221/images/multilingual-transformers-2.jpg)

In [2]:
import datasets 

ted = datasets.load_dataset('ted_multi', split='train')
ted

Generating train split:   0%|          | 0/258098 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6049 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7213 [00:00<?, ? examples/s]

Dataset ted_multi_translate downloaded and prepared to /root/.cache/huggingface/datasets/ted_multi_translate/plain_text/1.0.0/36fba834c6533853a24b6398207b3a1567455da505ceeed63bd94a5b7c6fd8b9. Subsequent calls will reuse this data.


Dataset({
    features: ['translations', 'talk_name'],
    num_rows: 258098
})

In [3]:
ted[0]

{'translations': {'language': ['ar',
   'bg',
   'de',
   'el',
   'en',
   'es',
   'eu',
   'fa',
   'fr',
   'fr-ca',
   'he',
   'hr',
   'hu',
   'it',
   'ja',
   'ko',
   'nb',
   'nl',
   'pl',
   'pt',
   'pt-br',
   'ro',
   'ru',
   'sq',
   'tr',
   'vi',
   'zh-cn',
   'zh-tw'],
  'translation': ['من ضمن جميع المثبطات المقلقة التي نعاني منها اليوم نفكر في المقام الاول في الامور المالية والاقتصادية واكثر ما يهمني بشكل اكثر هو عجز الحوار السياسي — قدرتنا على فهم الصراعات الحديثة على ماهي عليه , بالذهاب الى اصلها الفعلي وعلى فهم اللاعبين الرئيسيين وعلى التعامل معهم',
   'Наред с всички обезпокоителни дефицити , с които се сблъскваме днес - ние основно мислим за финансовите и икономическите - този , който ме безпокои най-вече е липсата на политически диалог - нашата способност да подходим към съвременните конфликти както те присъстват , да стигнем до източника на това , от което те произтичат и да разберем ключовите участници и да се разберем с тях .',
   'Unter den schwierige

In [4]:
idx = ted[0]['translations']['language'].index('en')
idx 

4

In [5]:
source = ted[0]['translations']['translation'][idx]
source

'Amongst all the troubling deficits we struggle with today — we think of financial and economic primarily — the ones that concern me most is the deficit of political dialogue — our ability to address modern conflicts as they are , to go to the source of what they &apos;re all about and to understand the key players and to deal with them .'

In [6]:
pairs = [] 

for i, translation in enumerate(ted[0]['translations']['translation']):
    if i != idx:
        pairs.append((source, translation))
        
pairs[0]

('Amongst all the troubling deficits we struggle with today — we think of financial and economic primarily — the ones that concern me most is the deficit of political dialogue — our ability to address modern conflicts as they are , to go to the source of what they &apos;re all about and to understand the key players and to deal with them .',
 'من ضمن جميع المثبطات المقلقة التي نعاني منها اليوم نفكر في المقام الاول في الامور المالية والاقتصادية واكثر ما يهمني بشكل اكثر هو عجز الحوار السياسي — قدرتنا على فهم الصراعات الحديثة على ماهي عليه , بالذهاب الى اصلها الفعلي وعلى فهم اللاعبين الرئيسيين وعلى التعامل معهم')

In [7]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=130431607886eb917eb60b1d55aef16e7433d16ca3d9d50dd63ceb8ac021a698
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [8]:
from sentence_transformers import InputExample 
from tqdm import tqdm 

lang_list = ['it', 'es', 'ar', 'fr', 'de']

train_samples = {f'en-{lang}': [] for lang in lang_list}

for row in tqdm(ted):
    idx = row['translations']['language'].index('en')
    source = row['translations']['translation'][idx].strip() 
    for i, lang in enumerate(row['translations']['language']):
        if lang in lang_list:
            translation = row['translations']['translation'][i].strip() 
            train_samples[f'en-{lang}'].append(source + '\t' + translation)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 258098/258098 [00:39<00:00, 6478.54it/s]


In [9]:
for lang_pair in train_samples.keys():
    print(f'{lang_pair}: {len(train_samples[lang_pair])}')

en-it: 204503
en-es: 196026
en-ar: 214111
en-fr: 192304
en-de: 167888


In [10]:
source + '\t' + translation

'( Applause )\t( Applausi )'

In [11]:
import gzip 
import os

if not os.path.exists('data'):
    os.mkdir('data')
    
for lang_pair in train_samples.keys():
    with gzip.open(f'data/ted-train-{lang_pair}.tsv.gz', 'wt', encoding='utf-8') as f:
        f.write('\n'.join(train_samples[lang_pair]))

In [12]:
from transformers import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [13]:
sentences = [
    'we will include several languages',
    '一些中文单词',
    'το ελληνικό αλφάβητο είναι πολύ ωραίο',
    'ჩვენ გვაქვს ქართული'
]

for text in sentences:
    print(bert_tokenizer.tokenize(text))

['we', 'will', 'include', 'several', 'languages']
['一', '[UNK]', '中', '文', '[UNK]', '[UNK]']
['τ', '##ο', 'ε', '##λ', '##λ', '##η', '##ν', '##ι', '##κ', '##ο', 'α', '##λ', '##φ', '##α', '##β', '##η', '##τ', '##ο', 'ε', '##ι', '##ν', '##α', '##ι', 'π', '##ο', '##λ', '##υ', 'ω', '##ρ', '##α', '##ι', '##ο']
['[UNK]', '[UNK]', '[UNK]']


In [14]:
from transformers import XLMRobertaTokenizer

xlmr_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

In [15]:
for text in sentences:
    print(xlmr_tokenizer.tokenize(text))

['▁we', '▁will', '▁include', '▁several', '▁language', 's']
['▁', '一些', '中文', '单', '词']
['▁το', '▁ελληνικό', '▁αλ', 'φά', 'βη', 'το', '▁είναι', '▁πολύ', '▁ωραίο']
['▁ჩვენ', '▁გვაქვს', '▁ქართული']


In [16]:
from sentence_transformers import models, SentenceTransformer

xlmr = models.Transformer('xlm-roberta-base')
pooling = models.Pooling(xlmr.get_word_embedding_dimension(), 
                         pooling_mode_mean_tokens=True)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
student = SentenceTransformer(modules=[xlmr, pooling])
student

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [18]:
teacher = SentenceTransformer('all-mpnet-base-v2')
teacher

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

### But here, there is a final normalization layer. We need to avoid outputting normalized embeddings for our student to mimic. So we either remove that normalization layer or use a model without it. The paraphrase models do not use normalization. We’ll use one of those.

In [19]:
teacher = SentenceTransformer('paraphrase-distilroberta-base-v2')
teacher

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [20]:
from sentence_transformers import ParallelSentencesDataset

data = ParallelSentencesDataset(
    student_model=student, 
    teacher_model=teacher, 
    batch_size=32, 
    use_embedding_cache=True
)

In [21]:
train_files = os.listdir('data')
train_files

['ted-train-en-it.tsv.gz',
 'ted-train-en-ar.tsv.gz',
 'ted-train-en-de.tsv.gz',
 'ted-train-en-es.tsv.gz',
 'ted-train-en-fr.tsv.gz']

In [22]:
for f in train_files:
    print(f)
    data.load_data('data/' + f, 
                   max_sentences=250_000, 
                   max_sentence_length=256)

ted-train-en-it.tsv.gz
ted-train-en-ar.tsv.gz
ted-train-en-de.tsv.gz
ted-train-en-es.tsv.gz
ted-train-en-fr.tsv.gz


In [23]:
from torch.utils.data import DataLoader 

loader = DataLoader(data, shuffle=True, batch_size=32)

In [24]:
from sentence_transformers.losses import MSELoss 

loss = MSELoss(model=student)

In [25]:
student.fit(
    train_objectives=[(loader, loss)],
    epochs=1,
    warmup_steps=int(len(loader) * 0.1),
    output_path='model1',
    optimizer_params={'lr': 4e-5, 'eps': 1e-6},
    save_best_model=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/56489 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:547: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  labels = torch.tensor(labels)


KeyboardInterrupt: 

In [26]:
import datasets

en = datasets.load_dataset('stsb_multi_mt', 'en', split='test')
en

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


Dataset({
    features: ['sentence1', 'sentence2', 'similarity_score'],
    num_rows: 1379
})

In [27]:
it = datasets.load_dataset('stsb_multi_mt', 'it', split='test')
it

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/it/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


Dataset({
    features: ['sentence1', 'sentence2', 'similarity_score'],
    num_rows: 1379
})

In [28]:
en[0]

{'sentence1': 'A girl is styling her hair.',
 'sentence2': 'A girl is brushing her hair.',
 'similarity_score': 2.5}

In [29]:
it[0]

{'sentence1': 'Una ragazza si acconcia i capelli.',
 'sentence2': 'Una ragazza si sta spazzolando i capelli.',
 'similarity_score': 2.5}

In [30]:
en = en.map(lambda x: {'similarity_score': x['similarity_score'] / 5.0})
it = it.map(lambda x: {'similarity_score': x['similarity_score'] / 5.0})

en[0], it[0]

  0%|          | 0/1379 [00:00<?, ?ex/s]

  0%|          | 0/1379 [00:00<?, ?ex/s]

({'sentence1': 'A girl is styling her hair.',
  'sentence2': 'A girl is brushing her hair.',
  'similarity_score': 0.5},
 {'sentence1': 'Una ragazza si acconcia i capelli.',
  'sentence2': 'Una ragazza si sta spazzolando i capelli.',
  'similarity_score': 0.5})

In [31]:
from sentence_transformers import InputExample

en_samples = []
it_samples = []
en_it_samples = []

for i in range(len(en)):
    en_samples.append(InputExample(
        texts=[en[i]['sentence1'], en[i]['sentence2']],
        label=en[i]['similarity_score']
    ))
    it_samples.append(InputExample(
        texts=[it[i]['sentence1'], it[i]['sentence2']],
        label=it[i]['similarity_score']
    ))
    en_it_samples.append(InputExample(
        texts=[en[i]['sentence1'], it[i]['sentence2']],
        label=en[i]['similarity_score']
    ))

In [32]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

en_eval = EmbeddingSimilarityEvaluator.from_input_examples(
    en_samples, write_csv=True
)
it_eval = EmbeddingSimilarityEvaluator.from_input_examples(
    it_samples, write_csv=True
)
en_it_eval = EmbeddingSimilarityEvaluator.from_input_examples(
    en_it_samples, write_csv=True
)

In [33]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer('model1')
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

In [34]:
en_eval(model)

0.8682219599128399

In [35]:
it_eval(model)

0.8409392982575622

In [36]:
en_it_eval(model)

0.8326430915501263